In [17]:
%matplotlib inline
from lxml import etree
from glob import glob
import tqdm
import numpy as np
from collections import defaultdict
import pandas as pd
from datetime import datetime as dt
import random
from collections import Counter
from joblib import Parallel, delayed

In [18]:

ns = {"pm":"http://www.politicalmashup.nl",
      "dp":"http://dilipad.history.ac.uk",
      'dc':'http://purl.org/dc/elements/1.1/'}

In [19]:
!ls ../../..

ChileanNewspapers GHI_course        KBRiR             Tatiana
DM                GenderPaper-3     LondonPulse       Untitled.ipynb
FB                GoldenAgents      ParliamentaryTime code_literacy
FUB               IMPRESSO          Review            uva-remaining
French_Revolution JDTHKeynote       SlovParl


In [20]:
proceedings = glob("../../../GenderPaper-3/data/proc/*.xml")
members = glob('../../../GenderPaper-3/data/members/*.xml')

In [21]:
to_date = lambda x: dt.strptime(x,'%Y-%m-%d')

In [23]:
def map_members_to_gender():
    mapping = {'male':0,'female':1}
    id2gender = defaultdict(int)
    for member in tqdm.tqdm_notebook(members):
        member_id = member.split('/')[-1].split('.xml')[0]
        
        with open(member,'rt') as member_xml:
            tree = etree.parse(member_xml)
        gender=tree.xpath(".//pm:gender",namespaces=ns)[0].text
        id2gender[member_id] = mapping[gender]
    return id2gender

id2gender = map_members_to_gender()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [24]:
def get_attrs(element,*args):
    return [element.get('{%s}'%ns['pm']+arg,'NaN') for arg in args]

def get_text(element):
    paragraphs = element.xpath('.//pm:p',namespaces=ns)
    return " ".join([etree.tostring(p,method="text",encoding="unicode").lower() for p in paragraphs])

def process_speech(speech):
    text = get_text(speech)
    el_id,member,party,function,role = get_attrs(speech,'id','member-ref','party-ref','function','role')
    return [el_id,len(text.split()),member,party,id2gender[member],function,role,text]

In [47]:
def process_document(proc):
        with open(proc,"rt") as input_xml: 
            tree = etree.parse(input_xml)
    
        debate_date = dt.strptime(tree.xpath('.//dc:date',namespaces=ns)[0].text,'%Y-%m-%d')
    
        topics = tree.xpath('.//pm:topic',namespaces=ns)
        
        data = []
        
        for topic in topics:   
            topic_title = get_attrs(topic,'title')
            
            if not topic_title:
                continue
            if 'abortion' in topic_title[0].lower():
                print(topic_title)
                topic_id = get_attrs(topic,'id')[0]
                #speakers2speeches = Counter(topic.xpath('.//pm:speech/@pm:member-ref',namespaces=ns))
                speeches = topic.xpath('.//pm:speech',namespaces=ns)
            
                for speech in speeches:
                    line = process_speech(speech)
                    #print(line)
                    if line:
                        #speech_count = speakers2speeches[line[3]]
                        line.extend([debate_date,topic_id])
                        data.append(line)
        if data:
            print(data)
            return data
        

In [48]:
proceedings_red = [p for p in proceedings if int(p.split('/')[-1].split('.')[3].split('-')[0]) > 1944]
#data = Parallel(n_jobs=4)(delayed(process_document)(p) for p in tqdm.tqdm_notebook(proceedings_red))
results = [process_document(p) for p in proceedings_red]

['ABORTION (AMENDMENT)']
[['uk.proc.d.1978-02-21.10.1.2', 1639, 'uk.m.16771', 'uk.p.Con', 0, 'NaN', 'mp', 'i beg to move, that leave be given to bring in a bill to make further provision with respect to the protection of the life of a viable foetus; to amend section 4 of the abortion act 1967; to regulate the provision of payment for consultation and advice in relation to the termination of pregnancy; and to make provision with respect to bodies corporate. i rise to ask leave to introduce this short bill to amend the abortion act 1967 by implementing three of the most important recommendations of the select committee of this house appointed in february 1975. in so doing i am providing the house with an opportunity to assert its will in a manner of major social and parliamentary importance.\n           as hon. members know, on every occasion that the issue of abortion law has been raised in recent years a majority has signified its wish for the law to be clarified and amended, only to f

['Orders of the Day — ABORTION LAW (REFORM) BILL']
[['uk.proc.d.1970-02-13.6.1.3', 1101, 'uk.m.17903', 'uk.p.Lab', 0, 'NaN', 'mp', 'i beg to move, that the bill be now read a second time.\n           this is the second occasion on which i have had the privilege of finding my name included in the list of successful candidates in the ballot for private members\' bills. as i have had the experience once before, it may be thought that i should be prepared for what will happen today. on the last occasion, shortly after my bill had been published, i was sent for by the under-secretary of state for the home department. my bill related to firearms in the countryside, and i was told that it was quite unacceptable to the government. another ten days went by and i was sent for by the minister of state, who told me in no uncertain terms that the first opinion which i had had from the government had been reinforced by subsequent thought.\n           i went away at the christmas recess in a state of

['ABORTION']
[['uk.proc.d.1975-10-21.9.1.1', 940, 'uk.m.14155', 'uk.p.Lab', 1, 'The Secretary of State for Social Services', 'government', "with permission, mr. speaker, i should like to make, in association with my right hon. friends the secretaries of state for scotland and wales, a statement on abortion. since march of this year a select committee has been considering the abortion (amendment) bill of my hon. friend the member for glasgow, pollok (mr. white). the committee made nine recommendations in its third special report to the house on 5th august, most of which follow up proposals made last year by the lane committee on the working of the abortion act. i am glad to tell the house that the government accept in principle all the select committee's recommendations.\n           my intentions, announced earlier this year, to strengthen the safeguards which apply to abortion in the private sector are similar to those recommended by the select committee. action has already been taken 

['ABORTION (AMENDMENT) BILL']
[['uk.proc.d.1979-11-01.8.1.1', 326, 'uk.m.22249', 'uk.p.Lab', 0, 'NaN', 'mp', "on a point of order, mr. speaker. i wish to raise a matter which i think will be of concern not only to the whole house but to the entire country.\n           yesterday morning the committee that is dealing with the abortion (amendment) bill was sitting in committee room 10. whatever one's views about that bill might be, there is widespread and growing public concern about it throughout the country. the consequence is that in that committee room the accommodation for the press and the public is completely inadequate, so much so that yesterday morning the press association representative and a large number of other press representatives were unable to get access. several members of the committee, on both sides, raised the matter with the chairman. he sought to give an undertaking that he would do something about the problem. however, looking at all of the committee rooms, one fi

["ABORTION ACT (MEMBER'S QUESTION)"]
[['uk.proc.d.1969-05-20.9.1.1', 9, 'uk.m.21885', 'uk.p.Lab', 0, 'chairman', 'chair', 'mr. st. john-stevas. to raise a point of order.\n        ', datetime.datetime(1969, 5, 20, 0, 0), 'uk.proc.d.1969-05-20.9'], ['uk.proc.d.1969-05-20.9.1.2', 173, 'uk.m.19173', 'uk.p.Con', 0, 'NaN', 'mp', 'mr. speaker, i wish to raise a point of order of which i have given you notice. it arises out of question time yesterday, when i addressed a question to the secretary of state for social services on the working of the abortion act.\n           it was then put to the secretary of state by the hon. member for manchester, exchange (mr. will griffiths) that in a widely publicised statement i had said that thousands of foreign women were flocking to london to use the facilities of the national health service made available by the abortion law. that statement was made without any attempt to check the facts, and i was rebuked by the secretary of state for making it.\n    

...

['ABORTION (AMENDMENT) BILL (SELECT COMMITTEE)']
[['uk.proc.d.1976-02-09.9.1.9', 1268, 'uk.m.16771', 'uk.p.Con', 0, 'NaN', 'mp', 'hon. members will recall that in the debate on the second reading of the abortion (amendment) bill on 7th february last year the minister of state. department of health and social security gave a firm undertaking that: should the eventuality arise that the house goes into a different session before the bill is considered, the government give the commitment to the sponsors of the bill that they will re-establish the select committee."—[ official report, 7th february 1975; vol. 885, c. 1794.] the motion now before the house honours that undertaking.\n           in the event, the select committee was unable to complete consideration of the bill before the end of the session. not that it was dilatory in any way; under the guidance of its chairman, the right hon member for sunderland, north (mr. willey)—i pay a warm tribute to his skilled, tactful, good-humoured 

['Partial-Birth Abortion']
['PARTIAL-BIRTH ABORTION']
[['uk.proc.d.1996-12-04.9.1.1', 1309, 'uk.m.22624', 'uk.p.Con', 1, 'NaN', 'mp', 'i beg to move, that leave be given to bring in a bill to prohibit partial-birth abortions. partial-birth abortion is defined in the bill as an abortion in which the person performing the abortion partially vaginally delivers a living foetus before killing the foetus and completing the delivery. the procedure, which originated in the united states, is usually used after the 20th week of pregnancy, and often much later. the practitioner, guided by ultrasound, pulls the living unborn child through the mother\'s vagina, except for the child\'s head, which is deliberately kept just within the uterus. the practitioner then puts surgical scissors or another medical instrument into the back of the skull, inserts a catheter and sucks out the brains.\n           in 1992, dr. martin haskell of dayton, ohio, wrote an eight-page paper explaining step by step how to 

['Abortion (Amendment) Bill']
[['uk.proc.d.1988-01-22.4.1.2', 573, 'uk.m.16469', 'uk.p.LDem', 0, 'NaN', 'mp', 'i beg to move, that the bill be now read a second time. in coming to the debate this morning, three months after i presented the bill, i thank those hon. members on both sides of the argument for the way in which, generally, the debate has been conducted. this is a difficult issue, which rouses sensitivities, anger, bitterness, and, sometimes, even hatred. by and large, members of this house have conducted the debate in the country with great dignity and decorum, giving this issue the respect to which it is entitled. i hope that we shall be able to mirror that in the house today.\n           fifty-nine years after the passage of the infant life preservation act and 20 years after the passage of the abortion act, it is my hope that the house will today allow this bill to proceed to commitee. public and parliamentary opinion clearly believes that, in the light of medical and sci

['ABORTION (AMENDMENT) BILL']
[['uk.proc.d.1974-06-14.14.1.3', 1, 'NaN', 'NaN', 0, 'NaN', 'mp', 'object.\n        ', datetime.datetime(1974, 6, 14, 0, 0), 'uk.proc.d.1974-06-14.14'], ['uk.proc.d.1974-06-14.14.1.4', 6, 'NaN', 'NaN', 0, 'chairman', 'chair', 'the motion is not agreed to.\n        ', datetime.datetime(1974, 6, 14, 0, 0), 'uk.proc.d.1974-06-14.14']]
['ABORTION (AMENDMENT) BILL']
['Orders of the Day — ABORTION (AMENDMENT) BILL']
['Orders of the Day — ABORTION (AMENDMENT) BILL']
[['uk.proc.d.1975-02-07.2.1.2', 252, 'uk.m.21715', 'uk.p.Con', 0, 'NaN', 'mp', "on a point of order, mr. deputy speaker. i should like to put to you, or perhaps through you to the minister, a point of order about today's business on which clarification might help the house. the fourth item on the order paper, proposing the appointment of a select committee, touches closely upon the private member's bill which the hon. member for glasgow, pollok (mr. white) will, i hope, shortly seek to move.\n        

['Orders of the Day — ABORTION (AMENDMENT) BILL']
[['uk.proc.d.1980-03-14.3.3.1', 109, 'uk.m.17319', 'uk.p.Lab', 0, 'NaN', 'mp', 'on a point of order, mr. speaker, i had understood that this morning we would be considering the final stages of the child maintenance orders (annual up—rating and exemption) bill, as amended in standing committee c. the house knows that that is an important bill. it is not surprising that i have raised this point of order. the bill concerns the responsibility of the secretary of state for single—parent families. it ensures that liable relatives fulfil their responsibilities.\n           no statement has been made by the minister, the leader of the house or the hon. member responsible for the bill. i do not understand why we are not discussing it—\n        ', datetime.datetime(1980, 3, 14, 0, 0), 'uk.proc.d.1980-03-14.3'], ['uk.proc.d.1980-03-14.3.3.2', 38, 'uk.m.22185', 'uk.p.Lab', 0, 'chairman', 'chair', 'order. i am sorry to interrupt the right hon. gentl

In [53]:
results = [i for i in results if i]

In [54]:
data_flattened = [l for batch in results for l in batch]
df = pd.DataFrame(data_flattened)

In [55]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,uk.proc.d.1978-02-21.10.1.2,1639,uk.m.16771,uk.p.Con,0,NaN,mp,"i beg to move, that leave be given to bring in...",1978-02-21,uk.proc.d.1978-02-21.10
1,uk.proc.d.1978-02-21.10.1.4,2,uk.m.21824,uk.p.Con,0,NaN,mp,rose —\n,1978-02-21,uk.proc.d.1978-02-21.10
2,uk.proc.d.1978-02-21.10.1.5,9,uk.m.22185,uk.p.Lab,0,chairman,chair,is the hon. gentleman rising to oppose the bil...,1978-02-21,uk.proc.d.1978-02-21.10
3,uk.proc.d.1978-02-21.10.1.6,1272,uk.m.21824,uk.p.Con,0,NaN,mp,"yes, mr. speaker.\n i oppose the mot...",1978-02-21,uk.proc.d.1978-02-21.10
4,uk.proc.d.1970-02-13.6.1.3,1101,uk.m.17903,uk.p.Lab,0,NaN,mp,"i beg to move, that the bill be now read a sec...",1970-02-13,uk.proc.d.1970-02-13.6


In [57]:

df.columns = ['id','word_count', # speech context
              'member','party','gender','function','role', # member attributes
              'text',
              'date', # document context
              'topic_id'] # topic context
print(df.shape)
df = df[(df.date > dt.strptime('1945-08-15','%Y-%m-%d')) & (df.date < dt.strptime('2013-05-08','%Y-%m-%d'))]
print(df.shape)

(2127, 10)
(2127, 10)


In [58]:
df.to_csv('../data/proc/abortion_debates.csv')